## Extract pixels along (a given) growth axis line

Extracting the ring measurements is a two-stage process, and, in this notebook,
the first stage is performed. To summarise, the pixel intensity values are
extracted along the axis of maximum growth from the segmented patches (the
predictions returned from the model). This gives a vector containing values
which correspond to, according to the neural network model, the probability of
a pixel belonging to a growth ring. Peakfinding, the second stage, is carried out
in the [next notebook](./3_-_Peakfinding_for_ring_widths.ipynb). As with the
predictions, growth line pixel extraction is carried out separately for the
LOIO and final experiments, although the methodology is identical between the
two. A square `k` by `k` patch, centred on a location along the growth axis, is
extracted from the model's predictions. The maximum value of this patch is
taken as the value assigned to the growth axis at the corresponding location.
This is repeated for all locations along the growth axis, and results in a
vector of pixel intensities (pseudo probability values of a pixel containing a
ring). Further details about this notebook, the files it reads and saves, as
well as all other aspects of the project can be found in the
[README](./README.md).

In [ ]:
import os
import glob
import tqdm.notebook
import yaml

import numpy as np
from shellai import preprocessing, tf_util

# load the config file
with open("config.yaml", "r") as fd:
    cfg = yaml.safe_load(fd)
print('Config file loaded')

In [ ]:
# # #  Directory settings
# base directory -- where we're store things to
dir_base = cfg['paths']['local']['base']

# directory containing the saved models and their corresponding raw predictions
dir_saved_model = cfg['paths']['local']['model']

# directory where we wish to save the extracted rings to
dir_saved_predictions = os.path.join(dir_base, "output")

# # # Pixel extraction settings
# Note that the pixel extraction works by, for each coordinate along the drawn
# mask, we take the mean/max value of a (N, N) sized patch. Empirically, we
# found the following values to work well

# method: "mean" or "max", using the mean or maximum value of the patch
ringfinding_method = "max"

# size of the patch from which to extract the ring
ringfinding_patch_size = 3

### Perform the extraction for the leave-one-image-out (LOIO) models
Note that we have to do this separately for both the LOIO models and the final model due to the different file and directory structure.

In [ ]:
# settings for the LOIO experiments

# location of the real images
base_image_dir = os.path.join(dir_base, cfg['images']['train_dir'])

# mask of what we want the predictions to be called
save_name_mask = "loo_{image_name}_all_predictions.npz"

# list of the images used in training. each one corresponds to a model where
# the model was trained on all but the named image.
image_names = cfg['images']['train_images']

# mask of the model predictions
pred_file_mask = 'model_checkpoint_*_predictions.npz'

In [ ]:
for image_name in tqdm.notebook.tqdm(image_names):
    save_path = os.path.join(
        dir_saved_predictions, 
        save_name_mask.format(image_name=image_name)
    )

    if os.path.exists(save_path):
        print(f'Already exists, skipping: {save_path:s}')
        continue

    # load the image and ground-truth segmentation
    image, _, ring_mask_full = preprocessing.load_image_data(
        base_image_dir, image_name
    )
    ring_mask = preprocessing.threshold_drawn_mask_and_skeletonize(
        ring_mask_full, sparse=False
    )
    # list of each model's predictions (different saved model per number of epochs)
    pred_paths = glob.glob(
        os.path.join(dir_saved_model, image_name, pred_file_mask)
    )

    # get the epoch numbers
    epochs = [
        int(os.path.basename(pred_path).split('_')[2])
        for pred_path in pred_paths
    ]

    # sort the paths by epoch number
    sorted_inds = np.argsort(epochs)
    epochs = [epochs[i] for i in sorted_inds]
    pred_paths = [pred_paths[i] for i in sorted_inds]

    # storage for saving
    ring_data = {"preds": {e: None for e in epochs}, "gt": None}

    # just to ignore bug
    patch_coords = flc = None

    for epoch, pred_path in tqdm.notebook.tqdm(
        zip(epochs, pred_paths), total=len(epochs), leave=False
    ):
        # load the predictions, coordinates along the drawn line, and coordinates
        # of each extracted patch
        with np.load(pred_path, allow_pickle=True) as fd:
            predictions = fd['predictions']
            flc = fd['full_line_coords']
            patch_coords = fd['patch_coords'] # [c0, c1, r0, r1]
            # patch_centres = fd['patch_centres'] # [idx0, idx1]
        
        # here, 'gt' refers to pixels extracted using the ground-truth segmentation
        if ring_data["gt"] is None:
            ring_data["gt"] = tf_util.extract_rings_patchbased(
                line_coords=flc,
                image=ring_mask, #type: ignore
                patch_size=ringfinding_patch_size,
                method=ringfinding_method
            )

        # create the prediction image and store it. this maps the predicted patches
        # into an image of a predefined shape
        predicted_mask, _ = tf_util.create_patch_image(
            patches=predictions,
            patch_coords=patch_coords,
            image_shape=image.shape
        )
        
        # as with the gt, extract the pixels along given coordinates
        ring_data["preds"][epoch] = tf_util.extract_rings_patchbased(
            line_coords=flc,
            image=predicted_mask,
            patch_size=ringfinding_patch_size,
            method=ringfinding_method
        )

    # save everything
    print(f'Saving: {save_path:s}')
    np.savez(
        save_path,
        ring_data=ring_data,
        image_name=image_name,
        epochs=epochs,
        flc=flc,
        patch_coordintes=patch_coords,
    )

## Repeat for the final model
Training of the final model was carried out using the ten training images from the LOIO experiments. Testing is carried out on images with no ground truth segmentation available and, therefore, no ground truth pixel values can be extracted.

In [ ]:
# settings for the LOIO experiments

# location of the real images
base_image_dir = os.path.join(dir_base, cfg['images']['test_dir'])

# mask of what we want the predictions to be called
save_name_mask = "final_{image_name}_all_predictions.npz"

# list of the images used in training. each one corresponds to a model where
# the model was trained on all but the named image.
image_names = cfg['images']['test_images']

# mask of the model predictions - note that segmentations were carried each set
# of model checkpoints, for each test image
pred_file_mask = 'model_checkpoint_*_predictions_{image_name:s}.npz'

In [ ]:
image_name = image_names[0]
save_path = os.path.join(
    dir_saved_predictions, 
    save_name_mask.format(image_name=image_name)
)
print(save_path)

with np.load(save_path, allow_pickle=True) as fd:
    print(fd.files)
    ring_data = fd['ring_data'].item()
    print(fd['image_name'])
    print(fd['epochs'])
    flc = fd['flc']
    patch_coordintes = fd['patch_coordintes']

In [ ]:
for image_name in tqdm.notebook.tqdm(image_names):
    save_path = os.path.join(
        dir_saved_predictions, 
        save_name_mask.format(image_name=image_name)
    )

    if os.path.exists(save_path):
        print(f'Already exists, skipping: {save_path:s}')
        continue

    # load the image and ground-truth segmentation
    image, _ = preprocessing.load_image_data(
        base_image_dir, image_name, no_rings=True
    )

    # list of each model's predictions (different saved model per number of epochs)
    pred_paths = glob.glob(
        os.path.join(
            dir_saved_model, 
            "final_model", 
            pred_file_mask.format(image_name=image_name)
        )
    )
    
    # get the epoch numbers
    epochs = [
        int(os.path.basename(pred_path).split('_')[2])
        for pred_path in pred_paths
    ]

    # sort the paths by epoch number
    sorted_inds = np.argsort(epochs)
    epochs = [epochs[i] for i in sorted_inds]
    pred_paths = [pred_paths[i] for i in sorted_inds]

    # storage for saving
    ring_data = {"preds": {e: None for e in epochs}, "gt": None}

    # just to ignore bug
    patch_coords = flc = None

    for epoch, pred_path in tqdm.notebook.tqdm(
        zip(epochs, pred_paths), total=len(epochs), leave=False
    ):
        # load the predictions, coordinates along the drawn line, and coordinates
        # of each extracted patch
        with np.load(pred_path, allow_pickle=True) as fd:
            predictions = fd['predictions']
            flc = fd['full_line_coords']
            patch_coords = fd['patch_coords'] # [c0, c1, r0, r1]
            # patch_centres = fd['patch_centres'] # [idx0, idx1]

        # create the prediction image and store it. this maps the predicted 
        # patches into an image of a predefined shape
        predicted_mask, _ = tf_util.create_patch_image(
            patches=predictions,
            patch_coords=patch_coords,
            image_shape=image.shape
        )
        
        # extract the pixels along given coordinates
        ring_data["preds"][epoch] = tf_util.extract_rings_patchbased(
            line_coords=flc,
            image=predicted_mask,
            patch_size=ringfinding_patch_size,
            method=ringfinding_method
        )

    # save everything
    print(f'Saving: {save_path:s}')
    np.savez(
        save_path,
        ring_data=ring_data,
        image_name=image_name,
        epochs=epochs,
        flc=flc,
        patch_coordintes=patch_coords,
    )